# Overview

*Why `HDF5`?*

## Setting up an `HDF5` database

*Minimal coded needed to do this goes here.  Would be nice if most of this is abstracted away and you just pass in some `.json` that specifies everything (i.e. where to look for the data, what files to use, etc.)*

## Training a network
*This is probably just a few lines?*

## Testing a network


*Hopefully it's just a matter of pointing the network to a db of stuff.  It should be easy to get a your error back for the set.  It might be also nice to get the filenames of the worst traces (maybe a method that takes a user-specified threshold and returns a list of files?)*